# Create and display Landsat Mosaics

## Installation

```
$ pip install requests mapboxgl
```

In [ ]:
import os
import json
import random
import requests
import datetime
import urllib.parse

from mapboxgl.utils import *
from mapboxgl.viz import *

token = os.environ["MAPBOX_ACCESS_TOKEN"]

endpoint = "https://landsatlive.live"


In [ ]:
bounds = [-6.17431640625, 42.79540065303723, 8.28369140625, 51.0275763378024]

geojson = {
    'type': 'Feature',
    'geometry': {
        'type': 'Polygon',
        'coordinates': [[
            [bounds[0], bounds[1]],
            [bounds[2], bounds[1]],
            [bounds[2], bounds[3]],
            [bounds[0], bounds[3]],
            [bounds[0], bounds[1]]]]},
    'properties': {}
}

In [ ]:
viz = LinestringViz(geojson,
    access_token=token,
    line_width_default=4,
    center=((bounds[0] + bounds[2]) / 2, (bounds[1] + bounds[3]) / 2),
    zoom=4,
)
viz.show()

# Create Mosaic and visualize highest NDVI value

In [ ]:
min_cloud = 0
max_cloud = 10

date_min="2019-01-01"
date_max="2019-06-19"
start = datetime.datetime.strptime(date_min, "%Y-%m-%d").strftime("%Y-%m-%dT00:00:00Z")
end = datetime.datetime.strptime(date_max, "%Y-%m-%d").strftime("%Y-%m-%dT23:59:59Z")

query = {
    "bbox": bounds,
    "time": f"{start}/{end}",
    "query": {
        "eo:sun_elevation": {"gt": 0},
        "landsat:tier": {"eq": "T1"},
        "collection": {"eq": "landsat-8-l1"},
        "eo:cloud_cover": {"gte": min_cloud, "lt": max_cloud}
    },
    "limit": 1000
}
r = requests.post(f"{endpoint}/mosaic/create", json=query, params = {"maximum_items_per_tile": 0})
results_l8 = r.json()

query_params = dict(
    expr="(b5-b4)/(b5+b4)",
    color_map="greens",
    rescale="-1,1",
    pixel_selection="highest",
)

qs = urllib.parse.urlencode(query_params)

tiles_url = results_l8["tiles"][0] + qs

viz = RasterTilesViz(
    tiles_url, 
    zoom=results_l8["minzoom"],
    access_token=token,
    tiles_size=256,
    tiles_bounds=results_l8["bounds"],
    center=results_l8["center"],
    tiles_minzoom=results_l8["minzoom"],
    tiles_maxzoom=results_l8["maxzoom"],
)
viz.show()

# Using rio-tiler Expression and Custom Pixel_Selection methods

### Create Mosaic and visualize True Color RGB with highest NDVI value

The idea is to use rio-tiler expression method to create a 4th band on which we'll base the pixel selection method.

The `lastband` method will: 
- analyse the stack of tiles
- find where the value for the 4th band is the highest
- construct a tile with the index obtained
- automatically remove the last band of the tile

In [ ]:
min_cloud = 0
max_cloud = 5

date_min="2018-01-01"
date_max="2019-12-31"
start = datetime.datetime.strptime(date_min, "%Y-%m-%d").strftime("%Y-%m-%dT00:00:00Z")
end = datetime.datetime.strptime(date_max, "%Y-%m-%d").strftime("%Y-%m-%dT23:59:59Z")

query = {
    "bbox": bounds,
    "time": f"{start}/{end}",
    "query": {
        "eo:sun_elevation": {"gt": 0},
        "landsat:tier": {"eq": "T1"},
        "collection": {"eq": "landsat-8-l1"},
        "eo:cloud_cover": {"gte": min_cloud, "lt": max_cloud}
    },
    "limit": 1000
}
r = requests.post(
    f"{endpoint}/mosaic/create",
    json=query,
    params = {
        "optimized_selection": False,
        "maximum_items_per_tile": 0,
        "seasons": "spring" # this should be a "," separated list of seasons
    }
)
results_l8 = r.json()

# Expression: "b4,b3,b2,(b5-b4)/(b5+b4)"
# rio-tiler will analyse this expression as
# band1: b4
# band2: b3
# band3: b2
# band4: (b5-b4)/(b5+b4) beeing the NDVI
query_params = dict(
    expr="b4,b3,b2,(b5-b4)/(b5+b4)",
    color_ops="gamma RGB 3.5, saturation 1.7, sigmoidal RGB 15 0.35", # Looks nice
    pixel_selection="lastband", # This is a custom pixel selection method
)

qs = urllib.parse.urlencode(query_params)

tiles_url = results_l8["tiles"][0] + qs

viz = RasterTilesViz(
    tiles_url, 
    zoom=results_l8["minzoom"],
    access_token=token,
    tiles_size=256,
    tiles_bounds=results_l8["bounds"],
    center=results_l8["center"],
    tiles_minzoom=results_l8["minzoom"],
    tiles_maxzoom=results_l8["maxzoom"],
)
viz.show()

###  Create Mosaic and visualize True Color RGB with highest mean NDVI,NDSI value

In [ ]:
min_cloud = 0
max_cloud = 5

date_min="2018-01-01"
date_max="2019-12-31"
start = datetime.datetime.strptime(date_min, "%Y-%m-%d").strftime("%Y-%m-%dT00:00:00Z")
end = datetime.datetime.strptime(date_max, "%Y-%m-%d").strftime("%Y-%m-%dT23:59:59Z")

query = {
    "bbox": bounds,
    "time": f"{start}/{end}",
    "query": {
        "eo:sun_elevation": {"gt": 0},
        "landsat:tier": {"eq": "T1"},
        "collection": {"eq": "landsat-8-l1"},
        "eo:cloud_cover": {"gte": min_cloud, "lt": max_cloud}
    },
    "limit": 1000
}
r = requests.post(
    f"{endpoint}/mosaic/create",
    json=query,
    params = {
        "optimized_selection": False,
        "maximum_items_per_tile": 0,
        "seasons": "spring" # this should be a "," separated list of seasons
    }
)
results_l8 = r.json()

# Expression: "b4,b3,b2,((b5-b4)/(b5+b4)+(b2-b5)/(b2+b5))/2"
# rio-tiler will analyse this expression as
# band1: b4
# band2: b3
# band3: b2
# band4: ((b5-b4)/(b5+b4)+(b2-b5)/(b2+b5))/2
# with (b5-b4)/(b5+b4) beeing the NDVI and (b2-b5)/(b2+b5) the NDSI
query_params = dict(
    expr="b4,b3,b2,((b5-b4)/(b5+b4)+(b2-b5)/(b2+b5))/2",
    color_ops="gamma RGB 3.5, saturation 1.7, sigmoidal RGB 15 0.35", # Looks nice
    pixel_selection="lastband", # This is a custom pixel selection method
)

qs = urllib.parse.urlencode(query_params)

tiles_url = results_l8["tiles"][0] + qs

viz = RasterTilesViz(
    tiles_url, 
    zoom=results_l8["minzoom"],
    access_token=token,
    tiles_size=256,
    tiles_bounds=results_l8["bounds"],
    center=results_l8["center"],
    tiles_minzoom=results_l8["minzoom"],
    tiles_maxzoom=results_l8["maxzoom"],
)
viz.show()